### 2章 介入効果を測るための回帰分析 

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as sm

In [2]:
# 1章で作成したbiased_data.csvを読込む
biased_data = pd.read_csv('biased_data.csv')
biased_data.head(3)

,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend,treatment
0,9,5) $500 - $750,675.83,1,0,Rural,1,Web,Mens E-Mail,0,0,0.0,1
1,9,5) $500 - $750,675.07,1,1,Rural,1,Phone,Mens E-Mail,0,0,0.0,1
2,2,2) $100 - $200,101.64,0,1,Urban,0,Web,Mens E-Mail,1,0,0.0,1


In [3]:
# 線形回帰
lr = LinearRegression()
x, y = biased_data[["history", "treatment"]], biased_data["spend"]
lr.fit(x, y)
# scikit-learn には summary出力はない

LinearRegression()

In [4]:
# summaryの見方参考サイト https://tanuhack.com/statsmodels-multiple-lra/
model = sm.ols(formula='spend ~ treatment + history', data=biased_data)
biased_fitted = model.fit()
biased_fitted.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  spend   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     20.52
Date:                Sun, 20 Sep 2020   Prob (F-statistic):           1.24e-09
Time:                        14:12:44   Log-Likelihood:            -1.3391e+05
No. Observations:               31925   AIC:                         2.678e+05
Df Residuals:                   31922   BIC:                         2.679e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3790      0.150      2.519      0.012       0.084       0.674
treatment      0.8637      0.182      4.750      0.000       0.507       1.220
history        0.0013      0.000      3.622      0.000       0.001       0.002
==============================================================================
Omnibus:                    69503.290   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        301492932.516
Skew:                          19.981   Prob(JB):                         0.00
Kurtosis:                     477.399   Cond. No.                         832.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [5]:
# coefのみ出力
biased_fitted.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,0.3790,0.150,2.519,0.012,0.084,0.674
treatment,0.8637,0.182,4.750,0.000,0.507,1.220
history,0.0013,0.000,3.622,0.000,0.001,0.002


In [6]:
# （参考）DataFrame型への変換
pd.read_html(biased_fitted.summary().tables[1].as_html(), header=0, index_col=0)[0]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,0.3790,0.150,2.519,0.012,0.084,0.674
treatment,0.8637,0.182,4.750,0.000,0.507,1.220
history,0.0013,0.000,3.622,0.000,0.001,0.002


### RCTデータでの回帰

In [7]:
# 1章で作成したmale_data.csvを読込む
male_data = pd.read_csv('male_data.csv')
male_data.head(3)

,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend,treatment
0,6,3) $200 - $350,329.08,1,1,Rural,1,Web,No E-Mail,0,0,0.0,0
1,9,5) $500 - $750,675.83,1,0,Rural,1,Web,Mens E-Mail,0,0,0.0,1
2,9,5) $500 - $750,675.07,1,1,Rural,1,Phone,Mens E-Mail,0,0,0.0,1


In [8]:
model = sm.ols(formula='spend ~ treatment + history', data=male_data)
male_fitted = model.fit()

In [9]:
print("バイアスなしデータの回帰")
display(male_fitted.summary().tables[1])

print("\n【比較】バイアスのあるデータの回帰")
display(biased_fitted.summary().tables[1])

バイアスなしデータの回帰


,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,0.3596,0.123,2.917,0.004,0.118,0.601
treatment,0.7674,0.145,5.285,0.000,0.483,1.052
history,0.0012,0.000,4.301,0.000,0.001,0.002



【比較】バイアスのあるデータの回帰


,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,0.3790,0.150,2.519,0.012,0.084,0.674
treatment,0.8637,0.182,4.750,0.000,0.507,1.220
history,0.0013,0.000,3.622,0.000,0.001,0.002


### 共変量 X として recency、channel、history を加える

In [10]:
model = sm.ols(formula='spend ~ treatment + recency + channel + history', data=biased_data)
add_cov_biased_fitted = model.fit()

In [11]:
print("バイアスなしデータの回帰")
display(male_fitted.summary().tables[1])

print("\n共変量を追加した、バイアスのあるデータの回帰")
display(add_cov_biased_fitted.summary().tables[1])

print("\nバイアスのあるデータの回帰")
display(biased_fitted.summary().tables[1])

バイアスなしデータの回帰


,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,0.3596,0.123,2.917,0.004,0.118,0.601
treatment,0.7674,0.145,5.285,0.000,0.483,1.052
history,0.0012,0.000,4.301,0.000,0.001,0.002



共変量を追加した、バイアスのあるデータの回帰


,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,0.4991,0.393,1.269,0.204,-0.272,1.270
channel[T.Phone],0.0359,0.316,0.114,0.910,-0.583,0.654
channel[T.Web],0.2967,0.315,0.942,0.346,-0.320,0.914
treatment,0.8099,0.186,4.353,0.000,0.445,1.175
recency,-0.0399,0.027,-1.476,0.140,-0.093,0.013
history,0.0012,0.000,3.161,0.002,0.000,0.002



バイアスのあるデータの回帰


,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,0.3790,0.150,2.519,0.012,0.084,0.674
treatment,0.8637,0.182,4.750,0.000,0.507,1.220
history,0.0013,0.000,3.622,0.000,0.001,0.002


### 脱落変数バイアス（OVB：Omitted Variable Bias）

### DataFrame化

In [12]:
# model_A 目的変数はspend、共変量Xのうちhistoryを含まない
# model_B 目的変数はspend、共変量Xは全て含む
# model_C 目的変数はhistory、treatmentも説明変数に加える、spendは説明変数に加えない

model_ABC = {
            'A':'spend ~ treatment + recency + channel',
            'B':'spend ~ treatment + recency + channel + history',
            'C':'history ~ treatment + recency + channel'
            }

In [13]:
# 空のデータフレームを作成
ovb_df = pd.DataFrame()

In [14]:
for i in model_ABC.keys():
    model = sm.ols(formula=model_ABC[i], data=biased_data)
    model_fitted = model.fit()
    temp_df = pd.read_html(model_fitted.summary().tables[1].as_html(), header=0, index_col=0)[0]
    temp_df["model"] = f"model_{i}"
    ovb_df = pd.concat([ovb_df, temp_df])

In [15]:
ovb_df

,coef,std err,t,P>|t|,[0.025,0.975],model
Intercept,1.2104,0.323,3.753,0.000,0.578,1.843,model_A
channel[T.Phone],-0.3399,0.292,-1.162,0.245,-0.913,0.233,model_A
channel[T.Web],-0.0764,0.292,-0.262,0.794,-0.648,0.496,model_A
treatment,0.8450,0.186,4.550,0.000,0.481,1.209,model_A
recency,-0.0574,0.026,-2.173,0.030,-0.109,-0.006,model_A
Intercept,0.4991,0.393,1.269,0.204,-0.272,1.270,model_B
channel[T.Phone],0.0359,0.316,0.114,0.910,-0.583,0.654,model_B
channel[T.Web],0.2967,0.315,0.942,0.346,-0.320,0.914,model_B
treatment,0.8099,0.186,4.353,0.000,0.445,1.175,model_B
recency,-0.0399,0.027,-1.476,0.140,-0.093,0.013,model_B


In [16]:
# モデルA、B、Cのtreatmentのパラメーターを抽出
treatment_coef = ovb_df.loc["treatment"]["coef"]
treatment_coef

treatment     0.8450
treatment     0.8099
treatment    28.4521
Name: coef, dtype: float64

In [17]:
# モデルB（目的変数はtreatment、すべての共変量を含む）からhistoryのパラメーターを抜き出す
history_coef = ovb_df[ovb_df["model"]=="model_B"].loc["history"]["coef"]
history_coef

0.0012

In [18]:
# OVBの確認
# model B の history パラメータ × model C のtreatmentパラメータ
history_coef * treatment_coef[2]

0.034142519999999996

In [19]:
# coef_gap
# model A の treatment パラメータ - model B の treament パラメータ
treatment_coef[0] - treatment_coef[1]

0.03510000000000002

In [20]:
# coef_gap と OVB はおおむね一致
# 共変量 historyを追加したことで、OVBが消失していることがわかる

### CIA (Conditional Independence Assumption) OVBがすべてゼロになる状態

In [21]:
# 問題点
# バイアスを評価できない
# 必要な共変量がデータにない

### 入れてはいけない変数

In [22]:
# visit は 購買に影響がある
# visit は メールに配信にも影響を受ける
# このような介入の影響を受ける辺ううを入れてはならない

In [23]:
model = sm.ols(formula='treatment ~ visit + channel + recency + history', data=biased_data)
visit_fitted = model.fit()
visit_fitted.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,0.7104,0.011,63.991,0.000,0.689,0.732
channel[T.Phone],-0.0633,0.009,-6.713,0.000,-0.082,-0.045
channel[T.Web],-0.0642,0.009,-6.828,0.000,-0.083,-0.046
visit,0.1520,0.008,19.954,0.000,0.137,0.167
recency,-0.0289,0.001,-36.447,0.000,-0.030,-0.027
history,0.0001,1.17e-05,9.680,0.000,9.02e-05,0.000


In [24]:
# visit は 相関がそれなりに大きいため、説明変数に加えたくなる

In [25]:
model = sm.ols(formula='spend ~ treatment + channel + recency + history + visit', data=biased_data)
visit_fitted = model.fit()
display(visit_fitted.summary().tables[1])

print("\n【比較】visitを含まない回帰")
display(add_cov_biased_fitted.summary().tables[1])

# visit を加えることで、treatmentの効果が 0.8099 から 0.1849 に下落

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,-0.4281,0.389,-1.101,0.271,-1.190,0.334
channel[T.Phone],0.1091,0.311,0.351,0.726,-0.501,0.719
channel[T.Web],0.1516,0.310,0.489,0.625,-0.457,0.760
treatment,0.1849,0.185,1.002,0.316,-0.177,0.547
recency,0.0102,0.027,0.381,0.703,-0.042,0.062
history,0.0007,0.000,1.758,0.079,-7.8e-05,0.001
visit,7.7126,0.253,30.511,0.000,7.217,8.208



【比較】visitを含まない回帰


,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,0.4991,0.393,1.269,0.204,-0.272,1.270
channel[T.Phone],0.0359,0.316,0.114,0.910,-0.583,0.654
channel[T.Web],0.2967,0.315,0.942,0.346,-0.320,0.914
treatment,0.8099,0.186,4.353,0.000,0.445,1.175
recency,-0.0399,0.027,-1.476,0.140,-0.093,0.013
history,0.0012,0.000,3.161,0.002,0.000,0.002
